# Airline Delay and Cancellation in US domestic flights
### Data Engineering Capstone Project

#### Project Summary

We have used data from the Bureau of Transportation Statistics of the US about the local flights.
We have built a data lake with Spark to extract and process the data. 


The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
!pip install -q us

In [2]:
import pandas as pd
from dateutil import tz
import datetime
import us
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour
from pyspark.sql.functions import weekofyear, date_format, minute, second, to_timestamp
from pyspark.sql.types import TimestampType
from pyspark.sql import functions as F

### Step 1: Scope the Project and Gather Data

#### Scope 
The objective of the project is to let the analysis of the US domestic flights delays and cancellations. For that, we have to take different data sources, build the data pipelines and the dimensional models required to be possible the analysis by the Data Analyst team.

#### Describe and Gather Data 
The following datasets have been used in the project:
- [Airline Delay and Cancellation Data, 2009 - 2018](https://www.kaggle.com/yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018): US domestic flights delays 
- [2015 Flight Delays and Cancellations](https://www.kaggle.com/usdot/flight-delays): data about airports and airlines

We are going to load the data about airlines, airports and flights:

In [3]:
airlines_df = pd.read_csv('flight_delays/airlines.csv')

In [4]:
airlines_df.head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [5]:
airlines_df.shape

(14, 2)

In [6]:
airports_df = pd.read_csv('flight_delays/airports.csv')

In [7]:
airports_df.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [8]:
airports_df.shape

(322, 7)

In [9]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

The number of flights is very big so by now, we are going to use only data from 2016.

In [10]:
flights_2016_df = spark.read.csv('flight_delays/2016.csv.gz', header=True)

In [11]:
flights_2016_df.limit(5).toPandas()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2016-01-01,DL,1248,DTW,LAX,1935,1935.0,0.0,23.0,1958.0,...,309.0,285.0,249.0,1979.0,None,None,None,None,None,None
1,2016-01-01,DL,1251,ATL,GRR,2125,2130.0,5.0,13.0,2143.0,...,116.0,109.0,92.0,640.0,None,None,None,None,None,None
2,2016-01-01,DL,1254,LAX,ATL,2255,2256.0,1.0,19.0,2315.0,...,245.0,231.0,207.0,1947.0,None,None,None,None,None,None
3,2016-01-01,DL,1255,SLC,ATL,1656,1700.0,4.0,12.0,1712.0,...,213.0,193.0,173.0,1590.0,None,None,None,None,None,None
4,2016-01-01,DL,1256,BZN,MSP,900,1012.0,72.0,63.0,1115.0,...,136.0,188.0,121.0,874.0,72.0,0.0,52.0,0.0,0.0,None


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
We will review the three datasets we are going to work with (airles, airports and flights). We will:
- check if there are missing values
- check if there are duplicates
- check if there are wrong or bad formed data (bad formed dates, negative distances,...)
- check possible inconsistence data in the calculated fields in flights.

#### airlines_df
This is a small dataset that it is possible to inspect directly

In [12]:
airlines_df

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


In [13]:
airlines_df['IATA_CODE'].nunique()

14

#### airports_df

In [14]:
airports_df.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [15]:
airports_df.shape

(322, 7)

In [16]:
airports_df[airports_df['COUNTRY'] != 'USA'].size

0

In [17]:
airports_df.isnull().sum()

IATA_CODE    0
AIRPORT      0
CITY         0
STATE        0
COUNTRY      0
LATITUDE     3
LONGITUDE    3
dtype: int64

In [18]:
airports_df.duplicated().sum()

0

In [19]:
print("Min lat: {} | Max lat: {}".format(airports_df['LATITUDE'].min(), airports_df['LATITUDE'].max()))

Min lat: 13.48345 | Max lat: 71.28545


In [20]:
print("Min lon: {} | Max lon: {}".format(airports_df['LONGITUDE'].min(), airports_df['LONGITUDE'].max()))

Min lon: -176.64603 | Max lon: -64.79856


In [21]:
states = airports_df['STATE'].unique()
states

array(['PA', 'TX', 'NM', 'SD', 'GA', 'MA', 'CA', 'NJ', 'AK', 'LA', 'NY',
       'IA', 'MI', 'CO', 'WI', 'NC', 'CT', 'ME', 'AL', 'MT', 'ND', 'MN',
       'WA', 'IL', 'TN', 'ID', 'PR', 'VT', 'MD', 'SC', 'OH', 'UT', 'VA',
       'WY', 'MO', 'WV', 'KY', 'FL', 'NV', 'OR', 'IN', 'AZ', 'AR', 'KS',
       'MS', 'NE', 'GU', 'HI', 'DE', 'OK', 'NH', 'AS', 'RI', 'VI'], dtype=object)

The airports_df dataset looks like right: the data is about US and, the states and values of latitude and longitude look ok, with only a few missing values.

#### flights_df

In [22]:
flights_df = flights_2016_df

In [23]:
flights_df.count()

5617658

In [24]:
flights_df.printSchema()

root
 |-- FL_DATE: string (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: string (nullable = true)
 |-- DEP_TIME: string (nullable = true)
 |-- DEP_DELAY: string (nullable = true)
 |-- TAXI_OUT: string (nullable = true)
 |-- WHEELS_OFF: string (nullable = true)
 |-- WHEELS_ON: string (nullable = true)
 |-- TAXI_IN: string (nullable = true)
 |-- CRS_ARR_TIME: string (nullable = true)
 |-- ARR_TIME: string (nullable = true)
 |-- ARR_DELAY: string (nullable = true)
 |-- CANCELLED: string (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: string (nullable = true)
 |-- CRS_ELAPSED_TIME: string (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: string (nullable = true)
 |-- AIR_TIME: string (nullable = true)
 |-- DISTANCE: string (nullable = true)
 |-- CARRIER_DELAY: string (nullable = true)
 |-- WEATHER_DELAY: strin

Let's check if there are duplicates and missing values:

In [25]:
if flights_df.count() > flights_df.dropDuplicates().count():
    raise ValueError('Data has duplicates')

In [26]:
flights_df_agg = flights_df.agg(
    *[F.count(F.when(F.isnull(c), c)).alias(c) for c in flights_df.columns])

In [27]:
flights_df_agg.toPandas().T

,0
FL_DATE,0
OP_CARRIER,0
OP_CARRIER_FL_NUM,0
ORIGIN,0
DEST,0
CRS_DEP_TIME,0
DEP_TIME,63456
DEP_DELAY,63456
TAXI_OUT,65418
WHEELS_OFF,65418


We are not going to use all the columns so, altough some of them have a lot of missing values (the last one for example) it is not going to be a problem but, we have to be sure we work only with rows with values in the important columns: 

In [28]:
flights_must_cols = ['FL_DATE', 'OP_CARRIER', 'ORIGIN', 'DEST', 
                     'CRS_DEP_TIME', 'DEP_DELAY', 'CRS_ARR_TIME', 
                     'ARR_DELAY', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME']

In [29]:
flights_df = flights_df.na.drop(subset=flights_must_cols)

In [30]:
flights_df.count()

5538145

A lot of missing values in the column 'CANCELLATION_CODE' but, that happens when a flight was not cancelled:

In [31]:
flights_df.groupby('CANCELLATION_CODE').count().show()

+-----------------+-------+
|CANCELLATION_CODE|  count|
+-----------------+-------+
|             null|5538145|
+-----------------+-------+



In [32]:
flights_df.groupby('CANCELLED').count().show()

+---------+-------+
|CANCELLED|  count|
+---------+-------+
|      0.0|5538145|
+---------+-------+



By definition: ACTUAL_ELAPSED_TIME = AIR_TIME + TAXI_IN + TAXI_OUT

In [33]:
flights_df.where(flights_df['ACTUAL_ELAPSED_TIME'] != 
                 (flights_df['AIR_TIME'] + 
                  flights_df['TAXI_IN'] + 
                  flights_df['TAXI_OUT'])).count()

0

In [34]:
flights_df.where(flights_df['DISTANCE'] <= 0).count()

0

In [35]:
flights_df.where(flights_df['AIR_TIME'] <= 0).count()

0

For non cancelled flights we don't have data about delay causes, which makes sense: 

In [36]:
flights_df.where(flights_df['CANCELLED'] == 1.0).\
    select('CANCELLATION_CODE',
           'CARRIER_DELAY',
           'WEATHER_DELAY',
           'NAS_DELAY',
           'SECURITY_DELAY',
           'LATE_AIRCRAFT_DELAY').limit(20).show()

+-----------------+-------------+-------------+---------+--------------+-------------------+
|CANCELLATION_CODE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|
+-----------------+-------------+-------------+---------+--------------+-------------------+
+-----------------+-------------+-------------+---------+--------------+-------------------+



In [37]:
flights_df.where((flights_df['CANCELLED']==1.0) & \
                 (flights_df['CARRIER_DELAY'].isNull()) & \
                 (flights_df['WEATHER_DELAY'].isNull()) & \
                 (flights_df['NAS_DELAY'].isNull()) & \
                 (flights_df['SECURITY_DELAY'].isNull()) & \
                 (flights_df['LATE_AIRCRAFT_DELAY'].isNull())).count()

0

The columns of *flights_df* which ends with '_TIME' (CRS_DEP_TIME, DEP_TIME, CRS_ARR_TIME, ARR_TIME) express a time in the format 'mmss' so, the value shouldn't be greater than 2359 (23:59)


In [38]:
flights_df.where((col('CRS_DEP_TIME').cast("integer") > 2359)).count()

0

In [39]:
flights_df.where((col('DEP_TIME').cast("integer") == 2400)).count()

527

In [40]:
flights_df.where((col('DEP_TIME').cast("integer") == 2400)).limit(5).toPandas()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2016-01-01,F9,566,DEN,RDU,2359,2400.0,1.0,21.0,21.0,...,185.0,183.0,157.0,1436.0,None,None,None,None,None,None
1,2016-01-01,B6,1503,JFK,SJU,2359,2400.0,1.0,20.0,20.0,...,221.0,221.0,197.0,1598.0,None,None,None,None,None,None
2,2016-01-01,B6,108,JFK,PWM,2245,2400.0,75.0,11.0,11.0,...,68.0,59.0,46.0,273.0,3.0,0.0,0.0,0.0,63.0,None
3,2016-01-02,AA,1230,SEA,DFW,5,2400.0,-5.0,13.0,13.0,...,230.0,233.0,208.0,1660.0,None,None,None,None,None,None
4,2016-01-03,OO,5728,LAX,SAT,2359,2400.0,1.0,58.0,58.0,...,173.0,197.0,135.0,1211.0,1.0,0.0,24.0,0.0,0.0,None


In [41]:
flights_df.where((col('DEP_TIME').cast("integer") > 2400)).count()

0

In [42]:
flights_df.where((col('CRS_DEP_TIME').cast("integer") == 2400)).count()

0

Let's explore the flights which were planned to departure close to the midnight:

In [43]:
flights_df.where((col('CRS_DEP_TIME').cast("integer") == 2359)).limit(5).toPandas()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2016-01-01,F9,710,DEN,RSW,2359,2359.0,0.0,17.0,16.0,...,211.0,200.0,177.0,1607.0,None,None,None,None,None,None
1,2016-01-01,F9,682,DEN,MCO,2359,3.0,4.0,20.0,23.0,...,206.0,196.0,169.0,1546.0,None,None,None,None,None,None
2,2016-01-01,F9,566,DEN,RDU,2359,2400.0,1.0,21.0,21.0,...,185.0,183.0,157.0,1436.0,None,None,None,None,None,None
3,2016-01-01,F9,300,DEN,TPA,2359,1.0,2.0,19.0,20.0,...,199.0,194.0,168.0,1506.0,None,None,None,None,None,None
4,2016-01-01,UA,1832,SFO,ORD,2359,2352.0,-7.0,13.0,5.0,...,252.0,237.0,218.0,1846.0,None,None,None,None,None,None


There are some time fields with not valid values so, we will remove that data:

In [44]:
flights_df = flights_df.where(
    (col('DEP_TIME').cast("integer") < 2400) & \
    (col('DEP_TIME').cast("integer") >= 0) & \
    (col('CRS_DEP_TIME').cast("integer") < 2400) & \
    (col('CRS_DEP_TIME').cast("integer") >= 0)
)

In that case, the arrival will be the following day. The columns which contains a time (CRS_DEP_TIME, DEP_TIME, CRS_ARR_TIME, ARR_TIME) should be padded to 4 chars so, something like '945' will be converted to '0945' which represents '09:45'

In [45]:
flights_final_df = flights_df.select(col('FL_DATE').alias('FL_DATE'),
                                     col('OP_CARRIER').alias('AIRLINE'),
                                     'ORIGIN',
                                     'DEST',
                                     col('OP_CARRIER_FL_NUM').\
                                         alias('FL_NUMBER'),
                                     F.lpad(col('CRS_DEP_TIME').cast("int"), 4, '0').\
                                         alias('CRS_DEP_DATETIME'),
                                     F.lpad(col('DEP_TIME').cast("int"), 4, '0').\
                                         alias('DEP_DATETIME'),
                                     col('DEP_DELAY').cast('int').alias('DEP_DELAY'),
                                     col('TAXI_OUT').cast("int"),
                                     F.lpad(col('WHEELS_OFF').cast("int"), 4, '0').\
                                         alias('WHEELS_OFF_DATETIME'),
                                     F.lpad(col('WHEELS_ON').cast("int"), 4, '0').\
                                         alias('WHEELS_ON_DATETIME'),
                                     col('TAXI_IN').cast("int"),
                                     F.lpad(col('CRS_ARR_TIME').cast("int"), 4, '0').\
                                         alias('CRS_ARR_DATETIME'),
                                     F.lpad(col('ARR_TIME').cast("int"), 4, '0').\
                                         alias('ARR_TIME_DATETIME'),
                                     col('ARR_DELAY').cast('int').alias('ARR_DELAY'),
                                    (col('CANCELLED')==1.0).alias('CANCELLED'),
                                     'CANCELLATION_CODE',
                                    (col('DIVERTED')==1.0).alias('DIVERTED'),                                     
                                     col('AIR_TIME').cast("int"),
                                     'DISTANCE',
                                     col('CRS_ELAPSED_TIME').cast("int").\
                                         alias('EXPECTED_ELAPSED_TIME'), 
                                     col('ACTUAL_ELAPSED_TIME').cast("int").\
                                         alias('ACTUAL_ELAPSED_TIME'))

In [46]:
flights_final_df.printSchema()

root
 |-- FL_DATE: string (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- FL_NUMBER: string (nullable = true)
 |-- CRS_DEP_DATETIME: string (nullable = true)
 |-- DEP_DATETIME: string (nullable = true)
 |-- DEP_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF_DATETIME: string (nullable = true)
 |-- WHEELS_ON_DATETIME: string (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- CRS_ARR_DATETIME: string (nullable = true)
 |-- ARR_TIME_DATETIME: string (nullable = true)
 |-- ARR_DELAY: integer (nullable = true)
 |-- CANCELLED: boolean (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: boolean (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: string (nullable = true)
 |-- EXPECTED_ELAPSED_TIME: integer (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: integer (nullable = true)



We have to build a udf function to build a datetime field from date and time fields. As the time fields are in the local timezone we will convert all the datetimes to UTC. We will use a broadcasted dictionary with the timezone of each airport:

In [47]:
states_tz = {}
for state in states:
    state_tz = us.states.lookup(state).time_zones[0]
    et = tz.gettz(state_tz)
    states_tz[state] = et

In [48]:
airport_tz = {}
for _, airport in airports_df.iterrows():
    airport_tz[airport['IATA_CODE']] = states_tz[airport['STATE']]

In [49]:
airport_tz_bc = spark.sparkContext.broadcast(airport_tz)

In [50]:
#@udf(returnType=TimestampType())
def convert_datetimes(expected_date, expected_time, airport, delay):
    """Provide a utc datetime from a date and a time in local airport timezone 

    Composes a datetime with expected_date and expected_time with the airport timezone,
    and add the delay, then converts that datetime to the UTC timezone.

    Args:
        expected_date (string): date
        expected_time (string): time
        airport (string): code of the airport in which timezone expected_time is.
        delay (int): delay in minutes to add 

    """    
    expected_datetime = datetime.datetime.strptime(expected_date + ' ' + expected_time, '%Y-%m-%d %H%M')  
    if airport in airport_tz_bc.value:
        airport_tz = airport_tz_bc.value[airport]
        real_datetime = expected_datetime + datetime.timedelta(minutes=delay)
        real_datetime_local_tz = real_datetime.replace(tzinfo=airport_tz)
        real_datetime_utc = real_datetime_local_tz.astimezone(datetime.timezone.utc) 
        result = real_datetime.replace(tzinfo=airport_tz)
    else:
        result = None 
    return result

In [51]:
convert_datetimes('2016-01-01', '2359', 'ABE', 1)

datetime.datetime(2016, 1, 2, 0, 0, tzinfo=tzfile('/usr/share/zoneinfo/America/New_York'))

In [52]:
convert_datetimes('2016-01-01', '2359', 'BAD_AIRPORT_CODE', 0)

We declare the function as a udf and try it on the fligh dataframe:

In [53]:
convert_datetimes_udf = udf(convert_datetimes, TimestampType())

In [54]:
flights_final_df.select(convert_datetimes_udf('FL_DATE', 
                                              'CRS_DEP_DATETIME', 
                                              'ORIGIN', 
                                              'DEP_DELAY')).limit(5).toPandas()

,"convert_datetimes(FL_DATE, CRS_DEP_DATETIME, ORIGIN, DEP_DELAY)"
0,2016-01-02 00:35:00
1,2016-01-02 02:30:00
2,2016-01-02 06:56:00
3,2016-01-02 00:00:00
4,2016-01-01 17:12:00


We will use this udf in the flight ETL.

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
The conceptual model has 5 entities: Flight, Airline, Airport, Datetime, Cancelation. 

<img src="./assets/model.jpg">

We will implement this model as a star schema that is very convinient for analysis. 

**Fact Tables**:
- Fligth

**Dimmension Tables**:
- Airline
- Airport
- Datetime
- Cancelation

#### 3.2 Mapping Out Data Pipelines
The ETL process has the following steps:

- We use Spark to load the raw data 
- We clean the data
- We transform the data (obtain calculated fields, drop useless columns)
- We save the final fact and dimension tables as parquet files

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
We will save the final parquet files here:

In [55]:
output_data = "/home/workspace/output/"

#### airline_table

In [56]:
airlines_df.columns = ['id_airline', 'name']

In [57]:
airline_table = spark.createDataFrame(airlines_df)

In [58]:
airline_table.show()

+----------+--------------------+
|id_airline|                name|
+----------+--------------------+
|        UA|United Air Lines ...|
|        AA|American Airlines...|
|        US|     US Airways Inc.|
|        F9|Frontier Airlines...|
|        B6|     JetBlue Airways|
|        OO|Skywest Airlines ...|
|        AS|Alaska Airlines Inc.|
|        NK|    Spirit Air Lines|
|        WN|Southwest Airline...|
|        DL|Delta Air Lines Inc.|
|        EV|Atlantic Southeas...|
|        HA|Hawaiian Airlines...|
|        MQ|American Eagle Ai...|
|        VX|      Virgin America|
+----------+--------------------+



In [59]:
airline_table.write.parquet(
    output_data + "airline_table.parquet",
    mode="overwrite"
)

#### airport_table

In [60]:
airports_df.columns = ['id_airport', 'name', 'city', 'state', 'country', 'latitude', 'longitude']

In [61]:
airports_df.head()

,id_airport,name,city,state,country,latitude,longitude
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [62]:
airport_table = spark.createDataFrame(airports_df)

In [63]:
airport_table.show(5)

+----------+--------------------+-----------+-----+-------+------------------+-------------------+
|id_airport|                name|       city|state|country|          latitude|          longitude|
+----------+--------------------+-----------+-----+-------+------------------+-------------------+
|       ABE|Lehigh Valley Int...|  Allentown|   PA|    USA|40.652359999999994|           -75.4404|
|       ABI|Abilene Regional ...|    Abilene|   TX|    USA|          32.41132|           -99.6819|
|       ABQ|Albuquerque Inter...|Albuquerque|   NM|    USA|          35.04022|-106.60918999999998|
|       ABR|Aberdeen Regional...|   Aberdeen|   SD|    USA|45.449059999999996|          -98.42183|
|       ABY|Southwest Georgia...|     Albany|   GA|    USA|          31.53552|          -84.19447|
+----------+--------------------+-----------+-----+-------+------------------+-------------------+
only showing top 5 rows



In [64]:
airport_table.write.parquet(
    output_data + "airport_table.parquet",
    mode="overwrite"
)

#### cancelation_table

In [65]:
cancelation_dict = {
    'id_cancelation': ['A', 'B', 'C', 'D'],
    'name': ['Airline/Carrier', 'Weather','National Air System', 'Security']}

cancelation_df = pd.DataFrame.from_dict(cancelation_dict,)

In [66]:
cancelation_table = spark.createDataFrame(cancelation_df)

In [67]:
cancelation_table.show()

+--------------+-------------------+
|id_cancelation|               name|
+--------------+-------------------+
|             A|    Airline/Carrier|
|             B|            Weather|
|             C|National Air System|
|             D|           Security|
+--------------+-------------------+



In [68]:
cancelation_table.write.parquet(
    output_data + "cancelation_table.parquet",
    mode="overwrite"
)

#### flight_table

In [69]:
flights_final_df.printSchema()

root
 |-- FL_DATE: string (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- FL_NUMBER: string (nullable = true)
 |-- CRS_DEP_DATETIME: string (nullable = true)
 |-- DEP_DATETIME: string (nullable = true)
 |-- DEP_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF_DATETIME: string (nullable = true)
 |-- WHEELS_ON_DATETIME: string (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- CRS_ARR_DATETIME: string (nullable = true)
 |-- ARR_TIME_DATETIME: string (nullable = true)
 |-- ARR_DELAY: integer (nullable = true)
 |-- CANCELLED: boolean (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: boolean (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: string (nullable = true)
 |-- EXPECTED_ELAPSED_TIME: integer (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: integer (nullable = true)



In [70]:
# We take a sample data because the full table is to big and it will take a lot of time.
# In production we would use the whole data and a lot of spark nodes
flights_final_df = flights_final_df.sample(False, 0.1, seed=0)

In [71]:
flight_table = flights_final_df.select(
    convert_datetimes_udf('FL_DATE', 
                          'CRS_DEP_DATETIME', 
                          'ORIGIN', 
                          F.lit(0)).alias('FL_DATETIME'),     
    'AIRLINE', 
    'ORIGIN', 
    'DEST', 
    'FL_NUMBER',                        
    'DEP_DELAY',                                          
    'ARR_DELAY',
    'CANCELLED',
    'CANCELLATION_CODE',
    'DIVERTED',
    'TAXI_OUT',
    'AIR_TIME',
    'TAXI_IN',
    'DISTANCE',
    'EXPECTED_ELAPSED_TIME',
    'ACTUAL_ELAPSED_TIME'                        
)

In [72]:
flight_table = flight_table.withColumn("FL_TIMESTAMP", F.unix_timestamp('FL_DATETIME'))

In [73]:
flight_table.limit(5).toPandas()

,FL_DATETIME,AIRLINE,ORIGIN,DEST,FL_NUMBER,DEP_DELAY,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,TAXI_OUT,AIR_TIME,TAXI_IN,DISTANCE,EXPECTED_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,FL_TIMESTAMP
0,2016-01-02 01:07:00,DL,ATL,JAX,1273,7,-2,False,None,False,10,45,3,270.0,67,58,1451696820
1,2016-01-02 00:20:00,DL,SEA,KOA,1287,1,-3,False,None,False,12,353,7,2688.0,376,372,1451694000
2,2016-01-01 19:50:00,DL,ATL,MSY,1293,0,-12,False,None,False,14,69,5,425.0,100,88,1451677800
3,2016-01-01 21:20:00,DL,SRQ,ATL,1297,-1,-7,False,None,False,16,71,7,444.0,100,94,1451683200
4,2016-01-01 15:30:00,DL,ATL,SDF,1302,1,0,False,None,False,12,56,7,321.0,76,75,1451662200


In [74]:
flight_table.printSchema()

root
 |-- FL_DATETIME: timestamp (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- FL_NUMBER: string (nullable = true)
 |-- DEP_DELAY: integer (nullable = true)
 |-- ARR_DELAY: integer (nullable = true)
 |-- CANCELLED: boolean (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: boolean (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- DISTANCE: string (nullable = true)
 |-- EXPECTED_ELAPSED_TIME: integer (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: integer (nullable = true)
 |-- FL_TIMESTAMP: long (nullable = true)



In [75]:
flight_table.drop('FL_DATETIME').write.parquet(
    output_data + "flight_table.parquet",
    mode="overwrite"
)

#### datetime_table

In [76]:
# Write code here
datetime_table = flight_table.select(
    'FL_TIMESTAMP',
    year('FL_DATETIME').alias('year'),    
    month('FL_DATETIME').alias('month'),    
    dayofmonth('FL_DATETIME').alias('day'),    
    hour('FL_DATETIME').alias('hour'),
    minute('FL_DATETIME').alias('minute'),    
    second('FL_DATETIME').alias('second'), 
    weekofyear('FL_DATETIME').alias('week'),
    date_format('FL_DATETIME', 'u').cast('int').alias('weekday')
).drop_duplicates()

In [77]:
datetime_table.limit(5).toPandas()

,FL_TIMESTAMP,year,month,day,hour,minute,second,week,weekday
0,1451648160,2016,1,1,11,36,0,53,5
1,1451770620,2016,1,2,21,37,0,53,6
2,1451737500,2016,1,2,12,25,0,53,6
3,1451847900,2016,1,3,19,5,0,53,7
4,1451839740,2016,1,3,16,49,0,53,7


In [78]:
datetime_table.count()

193229

In [79]:
datetime_table.write.parquet(
    output_data + "datetime_table.parquet",
    mode="overwrite"
)

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [80]:
assert flight_table.where(flights_df['ACTUAL_ELAPSED_TIME'] != \
                          (flights_df['AIR_TIME'] + 
                           flights_df['TAXI_IN'] + 
                           flights_df['TAXI_OUT'])).count() == 0

Let's check there are not bad aiport codes or bad airline codes in flight_table:

In [82]:
airline_codes = airline_table.select('id_airline').rdd.flatMap(lambda x: x).collect()

In [83]:
airline_codes

['UA',
 'AA',
 'US',
 'F9',
 'B6',
 'OO',
 'AS',
 'NK',
 'WN',
 'DL',
 'EV',
 'HA',
 'MQ',
 'VX']

In [84]:
assert flight_table.filter(~flight_table["AIRLINE"].isin(airline_codes)).count() == 0

In [85]:
airport_codes = airport_table.select('id_airport').rdd.flatMap(lambda x: x).collect()

In [86]:
flight_table.filter(~flight_table["ORIGIN"].isin(airport_codes)).count()

7

In [87]:
flight_table.filter(~flight_table["DEST"].isin(airport_codes)).count()

7

It looks like we have a very small number of flights with a wrong or new airport code as origin or destination. This could be a warning to review the official list of airports or to drop the rows

In [88]:
flight_table.filter(~flight_table["ORIGIN"].isin(airport_codes)).select("ORIGIN").distinct().show()

+------+
|ORIGIN|
+------+
|   PGD|
+------+



In [89]:
flight_table.filter(~flight_table["DEST"].isin(airport_codes)).select("DEST").distinct().show()

+----+
|DEST|
+----+
| PGD|
+----+



#### 4.3 Data dictionary 

These are the tables of the dimensional model with their fields and meaning:


**Fact Tables**:
- **Fligth**
    - **id_flight** int: unique incremental identifier, used in case of loading into redshift tables
    - **FL_TIMESTAMP** int: utc datetime of the scheduled departure as unix timestamp 
    - **AIRLINE** char: airline code of the flight
    - **ORIGIN** char: code of the state origin of the flight
    - **DEST** char: code of the destination state of the fligh
    - **FL_NUMBER** varchar: code of the number asigned to the flight
    - **DEP_DELAY** float: departure delay in minutes
    - **DEP_ARRAY** float: arrival delay in minutes
    - **CANCELLED** bool: indicates if the flight was cancelled
    - **CANCELLATION_CODE** char: indicates the cancellation cause, in case of cancellation
    - **DIVERTED** bool: indicates if the flight was diverted
    - **DISTANCE** float: distance between the two airports (in miles)
    - **TAXI_OUT** float: the time duration elapsed between departure from the origin airport gate and 'wheels off'
    - **AIR_TIME** float: the time duration between 'wheels off' and 'wheels on' time
    - **TAXI_IN** float: the time duration elapsed between 'wheels on' and gate arrival at the destination airport
    - **EXPECTED_ELAPSED_TIME** float: planned elapsed Time of flight, in minutes
    - **ACTUAL_ELAPSED_TIME** float: real elapsed Time of flight, in minutes (AIR_TIME+TAXI_IN+TAXI_OUT)

**Dimmension Tables**:
- **Airline**
    - **id_airline** char: code of the airline
    - **name** varchar: name of the airline
    

- **Airport**
    - **id_airport** char: code of the airport
    - **name** varchar: name of the airport
    - **city** varchar: city of the airport
    - **state** varchar: state of the airport
    - **country** varchar: country code of the airport
    - **latitude** float: latitude of the airport
    - **longitude** float: longitude of the airport


- **Datetime**:
    - **id_datetime** int: datetime timestamp
    - **year** int: year in number
    - **month** int: month in number
    - **day** int: day in number
    - **hour** int: hour in number
    - **minute** int: minutes in number
    - **second** int: seconds in number
    - **week** int: number of week of the year
    - **weekday** int: day of the week in number (1: Monday, ...)
  

- **Cancelation**
    - **id_cancelation** char: cancelation code
    - **name** varchar: description of the cancelation


#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

1. I have choosen to build a data lake with Spark to read and process the raw data and save the final model as parquet files. I assume that the objective of the Data Analyst team is to do monthly analysis so, it wouldn't be necessary to have a live Datawarehouse or a live Spark Cluster all the time: a process could run every month just to complete the ETL process of the new raw files and produce the parquet files, which could be stored in S3. When it were necessary, the Data Analysts could run a Spark cluster to read the clean parquet files and complete their analysis, or even to use a solution as Amazon Athena.  

2. The frequency would depend on the final users requirements. My assumpion is that these kind of reports will be for monthly statistics but, if the objective were to have an almost real time dashboard, it could be necessary to have a live Spark Cluster processing streaming data.

3. In the following scenarios:
    - **The data was increased by 100x**: the solution could be the same but, more nodes in the Spark cluster would be necessary in order to increase the spead
    - **The data populates a dashboard that must be updated on a daily basis by 7am every day**: it would be necessary to update the data with more frequency. In that case a tool like Airflow could be a good option to incorporate to monitorize all the process and to alert with its SLA features.
    - **The database needed to be accessed by 100+ people**: we could consider to use a MPP database like Redshift for these case. We could have as many nodes as we needed. Other option would be to increase the nodes of the Spark cluster. 